In [94]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# QBR

## Player stats

In [95]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome('C:/Users/bdtol/Downloads/chromedriver_win32/chromedriver.exe' ,chrome_options=options)

C:\Users\bdtol\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [122]:
final_titles=[]
player_url=[]
college_abr=[]
qbr=[]
paa=[]
plays=[]
epa=[]
passing =[]
run=[]
sack=[]
pen=[]
raw=[]
year = []
for i in range(2004,2020):
    
    url = "https://www.espn.com/college-football/qbr/_/season/{}".format(i)
    driver.get(url)
    
    #Clicks "Show More" on webpage twice
    c=0
    while c<2:
        more_buttons = driver.find_elements_by_class_name("AnchorLink.loadMore__link")
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        c+=1
    page_source = driver.page_source
    
    #Scrape the full page  
    soup = BeautifulSoup(page_source, 'html.parser')
    names = soup.find('tbody', class_="Table__TBODY")

    #Player BIO
    title = names.findAll('div')
    final_titles += [title[i].find('a').next_element for i in range(len(title))]
    player_url += [title[i].find('a').attrs['href'] for i in range(len(title))]
    college_abr += [title[i].find('span').next_element for i in range(len(title))]
    year.append([i for x in range(len(title))])
    
    #Player Stats
    stats = soup.find_all('tbody', class_="Table__TBODY")
    player_stats = stats[1].findAll('tr')
    qbr += [player_stats[i].findAll('td')[0].next_element for i in range(len(player_stats))]
    paa += [player_stats[i].findAll('td')[1].next_element for i in range(len(player_stats))]
    plays += [player_stats[i].findAll('td')[2].next_element for i in range(len(player_stats))]
    epa += [player_stats[i].findAll('td')[3].next_element for i in range(len(player_stats))]
    passing += [player_stats[i].findAll('td')[4].next_element for i in range(len(player_stats))]
    run += [player_stats[i].findAll('td')[5].next_element for i in range(len(player_stats))]
    sack += [player_stats[i].findAll('td')[6].next_element for i in range(len(player_stats))]
    pen += [player_stats[i].findAll('td')[7].next_element for i in range(len(player_stats))]
    raw += [player_stats[i].findAll('td')[8].next_element for i in range(len(player_stats))]

In [124]:
#breakdown nested list
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [125]:
df = pd.DataFrame([final_titles, college_abr, flatyear, qbr, paa, plays, epa, passing, run, sack, pen, raw])
df = pd.DataFrame.transpose(df)
df.columns = ['NAMES', 'COLLEGE', 'YEAR', 'QBR', 'PAA', 'PLAYS', 'EPA', 'PASS', 'RUN', 'SACK', 'PEN', 'RAW']

In [128]:
df.tail()

,NAMES,COLLEGE,YEAR,QBR,PAA,PLAYS,EPA,PASS,RUN,SACK,PEN,RAW
1919,Grant Loy,BGSU,2019,29.5,-16.7,334,-0.5,1.8,3.9,-4.9,-1.3,37.3
1920,Josh Adkins,NMSU,2019,28.5,-26.2,571,0.9,22.4,-7.0,-15.2,0.9,38.3
1921,Stone Smartt,ODU,2019,24.6,-22.0,294,-6.6,2.7,5.4,-16.6,1.8,31.5
1922,Kato Nelson,AKR,2019,20.9,-35.1,414,-11.1,6.3,2.5,-20.1,0.2,29.3
1923,Cephus Johnson III,USA,2019,16.4,-34.9,266,-18.1,-7.5,-2.7,-9.5,1.6,20.4


In [169]:
#df.to_csv('csv_files/player_stats.csv')

## Player bio

In [133]:
HTWT=[]
DOB=[]
draft=[]

for i in player_url:
    time.sleep(1)
    html_page = requests.get(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    r  = soup.findAll('div', class_='ttu')
    h=0
    d=0
    dr=0
    for e in r:
        if e.next_element == 'HT/WT':
            HTWT.append(e.next_element.next_element.next_element.next_element)
            h=1
        if e.next_element == 'DOB':
            DOB.append(e.next_element.next_element.next_element.next_element)
            d=1
        if e.next_element == 'Draft Info':
            draft.append(e.next_element.next_element.next_element.next_element)
            dr=1
            
    if h == 0:
        HTWT.append('na')
    if d == 0:
        DOB.append('na')
    if dr == 0:
        draft.append('na')

In [61]:
df = pd.read_csv('csv_files/player_stats.csv')

In [134]:
df2 = pd.DataFrame([HTWT, DOB, draft])
df2 = pd.DataFrame.transpose(df2)
df2.columns = ['HT/WT', 'DOB', 'DRAFT_STATUS']

In [139]:
df2.head()

,HT/WT,DOB,DRAFT_STATUS
0,na,6/7/1981,"2005: Rd 4, Pk 20 (CAR)"
1,"6' 4"", 215 lbs",5/7/1984 (36),"2005: Rd 1, Pk 1 (SF)"
2,na,na,na
3,na,12/31/1981,"2005: Rd 1, Pk 25 (WSH)"
4,na,4/22/1983,"2005: Rd 1, Pk 21 (JAX)"


In [140]:
df3 = pd.concat([df,df2], axis=1)

In [145]:
df3.to_csv('csv_files/player_stats.csv')

In [147]:
df3.head()

,NAMES,COLLEGE,YEAR,QBR,PAA,PLAYS,EPA,PASS,RUN,SACK,PEN,RAW,HT/WT,DOB,DRAFT_STATUS
0,Stefan LeFors,LOU,2004,86.3,68.3,351,66.8,66.5,2.8,-3.7,1.6,88.3,na,6/7/1981,"2005: Rd 4, Pk 20 (CAR)"
1,Alex Smith,UTAH,2004,84.5,75.8,480,73.4,63.5,16.0,-7.3,1.2,83.8,"6' 4"", 215 lbs",5/7/1984 (36),"2005: Rd 1, Pk 1 (SF)"
2,Jason White,OKLA,2004,84.1,56.6,454,61.2,70.9,-6.2,-6.1,2.7,78.5,na,na,na
3,Jason Campbell,AUB,2004,83.3,41.5,373,48.4,56.9,-3.9,-6.7,2.0,76.1,na,12/31/1981,"2005: Rd 1, Pk 25 (WSH)"
4,Matt Jones,ARK,2004,81.7,35.9,383,43.6,24.4,27.5,-7.7,-0.7,72.6,na,4/22/1983,"2005: Rd 1, Pk 21 (JAX)"


# Power Index

In [309]:
team_name = []
over=[]
off =[]
defe=[]
spec =[]
year=[]
for i in range(2005,2020):
    url = "https://www.espn.com/college-football/fpi/_/view/efficiencies/season/{}".format(i)
    driver.get(url)
    driver.set_window_size(100, 100)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    body = soup.find_all('tbody', class_= 'Table__TBODY')
    teams = body[0].findAll('div')


    team_name += [teams[i].findAll('a')[1].next_element for i in range(len(teams))]
    year.append([i for x in range(len(teams))])
    
    stats = body[1].findAll('tr')
    over += [stats[i].findAll('td')[0].next_element.next_element for i in range(len(stats))]
    off += [stats[i].findAll('td')[2].next_element.next_element for i in range(len(stats))]
    defe += [stats[i].findAll('td')[4].next_element.next_element for i in range(len(stats))]
    spec += [stats[i].findAll('td')[6].next_element.next_element for i in range(len(stats))]

In [310]:
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [311]:
power = pd.DataFrame([team_name, over, off, defe, spec, flatyear])
power = pd.DataFrame.transpose(power)
power.columns = ['COLLEGE', 'OVERALL', 'OFFENSE', 'DEFENSE', 'SPECIAL', 'YEAR']

In [339]:
power.to_csv('csv_files/power_index.csv')

# Passing stats